In [2]:
from clique_benchmark import Trials
from graph_database import Graphs, Graph
graph_db = Graphs()
trials = Trials()

In [7]:
# run this cell to load the job results for all trials that are waiting pending job results
trials = Trials()
await trials.load_results()

In [11]:
print(trials.get(graph_id=74)[0].graph.g)
print(trials.get(graph_id=74)[0].clique_size)
print(trials.get(graph_id=74)[0].grover_iterations)
print(graph_db.get(graph_id=74)[0].clique_counts)

011011
2
2
[16, 9, 5, 1, 0]


In [4]:
print(sorted(trials.get(graph_id=74)[0].counts.items(), key=lambda t: t[1], reverse=True))
print(sorted(trials.get(graph_id=74)[0].simulation_counts.items(), key=lambda t: t[1], reverse=True))

[('0101', 715), ('0111', 710), ('1101', 670), ('1100', 665), ('1000', 657), ('0000', 654), ('0100', 646), ('1010', 639), ('1111', 631), ('0110', 630), ('1110', 618), ('0011', 596), ('0010', 573), ('1011', 563), ('1001', 526), ('0001', 507)]
[('0101', 2050), ('1001', 1916), ('1100', 1892), ('1010', 1856), ('1101', 1852), ('0000', 52), ('0100', 48), ('1111', 47), ('0010', 44), ('0011', 43), ('1000', 40), ('1011', 40), ('1110', 37), ('0001', 31), ('0111', 28), ('0110', 24)]


In [10]:
print(trials.get(graph_id=51376)[0].graph.g)
print(trials.get(graph_id=51376)[0].clique_size)
print(trials.get(graph_id=51376)[0].grover_iterations)
print(graph_db.get(graph_id=51376)[0].clique_counts[2])
print(sorted(trials.get(graph_id=51376)[0].counts.items(), key=lambda t: t[1], reverse=True))
print(sorted(trials.get(graph_id=51376)[0].simulation_counts.items(), key=lambda t: t[1], reverse=True))

1111101111
2
1
18
[('11000', 350), ('10100', 331), ('01001', 329), ('10010', 328), ('01010', 327), ('01000', 326), ('00110', 325), ('11100', 325), ('10000', 318), ('10110', 318), ('00011', 318), ('01100', 316), ('11111', 315), ('00100', 315), ('10011', 313), ('00101', 312), ('10001', 311), ('11101', 310), ('11011', 310), ('10101', 308), ('01110', 307), ('01011', 307), ('11110', 304), ('11010', 303), ('01101', 302), ('11001', 302), ('10111', 299), ('00010', 298), ('00111', 297), ('01111', 296), ('00000', 293), ('00001', 287)]
[('00100', 353), ('11011', 351), ('10101', 343), ('10111', 334), ('01000', 334), ('00011', 331), ('11111', 329), ('10010', 327), ('10001', 326), ('11101', 322), ('11010', 316), ('01111', 316), ('10100', 314), ('00101', 312), ('11100', 311), ('10000', 310), ('01110', 309), ('01011', 306), ('01001', 306), ('10110', 306), ('00010', 305), ('11000', 305), ('00001', 303), ('01100', 301), ('10011', 300), ('11001', 300), ('11110', 298), ('00110', 296), ('00000', 296), ('01